In this trial, I am performing Domain Adaptatation Techniques.

In [158]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Model
import time
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import keras as K
from tensorflow import keras
#from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from keras.layers import Dense


In [159]:

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [160]:
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [161]:
# Convert to grayscale by taking the average of RGB channels
train_images = np.mean(train_images, axis=-1, keepdims=True)
test_images = np.mean(test_images, axis=-1, keepdims=True)

In [162]:
#normalizing the datasets in the range of 0-1
train_images = train_images/255.0
test_images = test_images/255.0

Defining our base  model

The model below is an inspiration from tensorflow's functional models such as VGG16

In [163]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization


def base_model(input_shape):
    base_model = Sequential()
    
    base_model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(input_shape)))
    base_model.add(BatchNormalization())
    base_model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
    base_model.add(BatchNormalization())
    base_model.add(MaxPooling2D(pool_size=(2,2)))
    base_model.add(Dropout(0.25))

    base_model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    base_model.add(BatchNormalization())
    base_model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    base_model.add(BatchNormalization())
    base_model.add(MaxPooling2D(pool_size=(2,2)))
    base_model.add(Dropout(0.25))
    base_model.add(Dense(64, activation='relu', name='dense_1'))
    
    return base_model


Defining our Domain Classifier

In [164]:
def domain_classifier(input_shape):
    # model = tf.keras.Sequential([
    #     layers.Dense(128, activation='relu', input_shape=input_shape),
    #     layers.Dense(1, activation='sigmoid')
    # ])
    dClassify = Sequential()
   #dClassify.add(Flatten())
    dClassify.add(Dense(64, activation='relu', name='dense_2',input_shape = (input_shape)))
    dClassify.add(Dense(10, activation='softmax')) 
    
    return dClassify


DANN (Domain-Adversarial Neural Networks)

In [165]:
def dann_model(base_model, domain_classifier):
    base_model.trainable = True
    domain_classifier.trainable = True

    input_image = layers.Input(shape=(input_shape))

    # Feature extractor from base model
    feature = base_model(input_image)

    # Domain classifier
    domain_output = domain_classifier(feature)

    dann = Model(inputs=input_image, outputs=[feature, domain_output])
    return dann


In [166]:
input_shape = train_images.shape[1:]
num_classes = 10

In [167]:
# Create an instance of the base model
base_model_instance = base_model(input_shape)

# Create an instance of the domain classifier
domain_classifier_instance = domain_classifier((64,))  # Assuming the base model outputs 128-dimensional features

# Create an instance of the DANN model
dann = dann_model(base_model_instance, domain_classifier_instance)

# Create domain labels for source and target data
source_domain_labels = np.ones((len(train_images), 1))
target_domain_labels = np.zeros((len(target_images), 1))
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
dann.compile(optimizer=opt,
              metrics=['accuracy'])

In [168]:
dann.layers

In [169]:
# Train the model and display the activations after each epoch
start_time = time.time()
dann.fit(train_images,[train_labels,source_domain_labels],
         epochs=1,
         validation_data=(test_images, [test_labels,target_domain_labels]))
end_time = time.time()

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_t

In [144]:
test_loss, test_acc = dann.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1972, in test_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1956, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1944, in run_step  **
        outputs = model.test_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1850, in test_step
        y_pred = self(x, training=False)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_tra

In [49]:
base_model.layers

Now, I will be performing JTL (Joint Transfer Learning) with FMNIST datasets 

In [155]:
# Load Fashion MNIST dataset
(target_images, target_labels), (test0_images, test0_labels) = datasets.fashion_mnist.load_data()


In [156]:
test0_images.shape

(10000, 28, 28)

In [157]:
target_images = target_images.reshape((60000,28,28,1))
target_images = target_images/255.0

test0_images = test0_images.reshape((10000,28,28,1))
test0_images = test0_images/255.0

DANN (Domain-Adversarial Neural Networks)

In [55]:
for i in range(2):
    base_model.pop()
    
#here we are using Feature-based Domain Adaptation Technique
# here we try to distinguish between source and target domain data, forcing the model to learn domain-invariant features by adding and training new TOP_LAYERS


# A Dense classifier - ten classes
base_model.add(Dense(64, activation='relu', name='dense_1'))
base_model.add(Dense(10, activation='softmax'))

In [56]:
base_model.layers

In [57]:
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
base_model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
# Train the model and display the activations after each epoch
start_time = time.time()
base_model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels))
end_time = time.time()

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_t

In [ ]:
test_loss, test_acc = base_model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

Now, I will be performing JTL (Joint Transfer Learning) with SVHN datasets

SVHN (Street View House Numbers): This dataset contains images of house numbers collected from Google Street View. It has 10 classes (0-9) and is somewhat similar to CIFAR-10 in terms of its small, cropped images.

In [ ]:
from keras.datasets import svhn

# Load the SVHN dataset
(train_images, train_labels), (test_images, test_labels) = svhn.load_data()

In [ ]:
train_images = train_images.reshape((6000,28,28,1))
train_images = train_images/255.0

test_images = test_images.reshape((54000,28,28,1))
test_images = test_images/255.0

In [121]:
from sklearn.model_selection import train_test_split

# Load Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()

# Split the training data into a 50:50 train-test split
#train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.7, random_state=42)

In [122]:
# Convert to grayscale by taking the average of RGB channels
train_images = np.mean(train_images, axis=-1, keepdims=True)
test_images = np.mean(test_images, axis=-1, keepdims=True)

In [123]:
train_images = train_images/255.0
test_images = test_images/255.0

In [95]:
test_images.shape

(10000, 32, 32, 1)

In [68]:
#reusing the saved model
import tensorflow as tf
from tensorflow import keras

loaded_model = keras.models.load_model('model_cifar.h5')


In [62]:
for l in loaded_model.layers:
    print(l)
    

In [74]:
# from tensorflow.keras.layers import Dense,Flatten
# from tensorflow.keras import Model

In [69]:
#making changes in the layers of the pre-saved model
for i in range(2):
    loaded_model.pop()

#here we are using the same lower layer layers of the initial model but adding a new top fully connected layers
#so as to adapt with the new target domain

# x = loaded_model.layers[-3].output 
# x = Flatten()(x)
# x = Dense(64,activation='relu',name='dense_1')
# predictions = Dense(10, activation = "softmax")(x)
# final_model = Model(inputs = loaded_model.input, outputs = predictions)

In [70]:
# #adding new flatten and dense layers to help the model adapt the target domain
loaded_model.add(tf.keras.layers.Flatten(name='flatten'))
loaded_model.add(tf.keras.layers.Dense(64, activation='relu', name='dense_1'))
loaded_model.add(tf.keras.layers.Dense(100, activation='softmax', name='output'))
# #loaded_model.add(tf.keras.layers.Dense(128, activation='relu', name='dense_2'))


In [65]:
for l in loaded_model.layers:
    print(l)

In [43]:
#here we freeze the convolutional layers so that its weights and biases are not affected
#we only want to have the top fully connected adjusted as per the target
for layer in loaded_model.layers[:5]:
    layer.trainable = False

In [71]:
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
loaded_model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [72]:

# Train the model and display the activations after each epoch
start_time = time.time()
loaded_model.fit(train_images, train_labels, epochs=3, validation_data=(test_images, test_labels))
end_time = time.time()


Epoch 1/3
469/469 [==============================] - 45s 96ms/step - loss: 4.1889 - accuracy: 0.0759 - val_loss: 3.7594 - val_accuracy: 0.1390
Epoch 2/3
469/469 [==============================] - 46s 98ms/step - loss: 3.3958 - accuracy: 0.2083 - val_loss: 3.5536 - val_accuracy: 0.1754
Epoch 3/3
469/469 [==============================] - 45s 97ms/step - loss: 2.7791 - accuracy: 0.3209 - val_loss: 3.6499 - val_accuracy: 0.1814


In [50]:
test_loss, test_acc = loaded_model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

1407/1407 [==============================] - 51s 36ms/step - loss: 4.5288 - accuracy: 0.1516
Test accuracy: 0.15164443850517273
Time elapsed:  580.2481989860535
